<a href="https://colab.research.google.com/github/Malanidhruv/Python-projects/blob/main/backtesting_for_BankNifty_low%26high.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pya3
%pip install nsetools

In [ ]:
from pya3 import *
from datetime import datetime, timedelta
from nsetools import Nse
import holidays
from google.colab import drive
import pandas as pd

def get_user_credentials():
    """Prompt the user for their credentials and return them."""
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice(user_id, api_key):
    """Initialize the AliceBlue session."""
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()  # Log in to AliceBlue
    return alice

def get_market_holidays(year):
    """Return a list of market holidays for the given year."""
    market_holidays = holidays.India(years=year)
    market_holidays.append(datetime(2024, 3, 8).date())  # Mahashivratri
    return market_holidays

def find_previous_trading_day(current_date, market_holidays):
    """Find the previous trading day, excluding weekends and holidays."""
    previous_trading_day = current_date - timedelta(days=1)
    while previous_trading_day.weekday() >= 5 or previous_trading_day in market_holidays:
        previous_trading_day -= timedelta(days=1)
    return previous_trading_day

def fetch_historical_data(alice, instrument, date):
    """Fetch historical data for the given instrument on the specified date."""
    from_datetime = datetime.combine(date, datetime.min.time())
    to_datetime = datetime.combine(date, datetime.max.time())
    interval = "D"
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    return historical_data

def calculate_strike_prices(closing_price):
    """Calculate call and put option strike prices based on the closing price."""
    ce_strike_price = int(closing_price / 100) * 100
    pe_strike_price = ce_strike_price + 100
    return ce_strike_price, pe_strike_price

def construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price):
    """Construct option symbols for call and put options."""
    symbol_ce = f'BANKNIFTY{expiry_date.strftime("%d%b%y").upper()}C{ce_strike_price}'
    symbol_pe = f'BANKNIFTY{expiry_date.strftime("%d%b%y").upper()}P{pe_strike_price}'
    return symbol_ce, symbol_pe

def fetch_highest_high(alice, symbol, from_datetime, to_datetime, interval):
    """Fetch the highest high for the given symbol."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    highest_high = historical_data['high'].max()
    return round(float(highest_high))

def fetch_lowest_low(alice, symbol, from_datetime, to_datetime, interval):
    """Fetch the lowest low for the given symbol."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    lowest_low = historical_data['low'].min()  # Use min() to get the lowest low
    return round(float(lowest_low))

def fetch_current_data(alice, symbol, from_time, to_time):
    """Fetch current data for the given symbol within the specified time range."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    current_data = alice.get_historical(instrument, from_time, to_time, interval='1')
    return current_data

def place_order(alice, transaction_type, symbol, price_value, quantity=25):
    """Place an order for the given symbol."""
    if not isinstance(price_value, (float, int)):
        raise ValueError(f"Price must be a float or int, got {type(price_value).__name__}")

    price_value = float(price_value)  # Ensure price_value is a float

    return alice.place_order(
        transaction_type=transaction_type,
        instrument=alice.get_instrument_by_symbol('NFO', symbol),
        quantity=quantity,
        order_type=OrderType.StopLossLimit,
        product_type=ProductType.BracketOrder,
        price=price_value,
        trigger_price=price_value,
        stop_loss=75.0,
        square_off=150.0,
        trailing_sl=0.0,
        is_amo=True,
        order_tag='order1'
    )

def main():
    # Step 1: Mount Google Drive
    drive.mount('/content/drive')

    # Get user credentials and initialize AliceBlue
    user_id, api_key = get_user_credentials()
    alice = initialize_alice(user_id, api_key)

    # Create an instance of Nse
    nse = Nse()

    # Get market holidays for the current year
    current_date = datetime.now()
    market_holidays = get_market_holidays(current_date.year)

    # Find the previous trading day
    previous_trading_day = find_previous_trading_day(current_date, market_holidays)
    print(f"Previous Trading Day: {previous_trading_day}")

    # Fetch the closing price of the future contract
    instrument = alice.get_instrument_by_symbol('NFO', 'BANKNIFTY24DEC24F')
    historical_data = fetch_historical_data(alice, instrument, previous_trading_day)

    if historical_data.empty:
        print("No historical data found for the future contract.")
        return

    last_close_price = historical_data['close'].iloc[-1]
    print(f"Closing Price of Future Contract: {last_close_price}")

    # Calculate strike prices based on the closing price of the future contract
    ce_strike_price, pe_strike_price = calculate_strike_prices(last_close_price)

    # Define the expiry date for options
    expiry_date = datetime(2024, 12, 24)

    # Construct option symbols
    symbol_ce, symbol_pe = construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price)
    print("CE Option Symbol:", symbol_ce)
    print("PE Option Symbol:", symbol_pe)


    # Define from_datetime, to_datetime, and interval for fetching highs and lows
    interval = "1"  # ["1", "D"]
    # Fetch and print current data for CE and PE from 9:15:29 to 15:17:29
    from_time = datetime.combine(current_date, time(9, 15, 29))
    to_time = datetime.combine(current_date, time(15, 17, 29))

    print("Fetching current data for CE and PE options...")
    ce_current_data = fetch_current_data(alice, symbol_ce, from_time, to_time)
    pe_current_data = fetch_current_data(alice, symbol_pe, from_time, to_time)

    print(ce_current_data)

     # Define from_datetime, to_datetime, and interval for fetching highs and lows
    from_datetime = datetime.combine(previous_trading_day, datetime.min.time())
    to_datetime = datetime.combine(previous_trading_day, datetime.max.time())
    interval = "D"

    # Get the highest high and place orders for CE
    highest_high_ce = fetch_highest_high(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Highest High CE Value:", highest_high_ce)
    print(place_order(alice, TransactionType.Sell, symbol_ce, highest_high_ce))

    # Get the highest high and place orders for PE
    highest_high_pe = fetch_highest_high(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Highest High PE Value:", highest_high_pe)
    print(place_order(alice, TransactionType.Sell, symbol_pe, highest_high_pe))

    # Get the lowest low and place orders for CE
    lowest_low_ce = fetch_lowest_low(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low CE Value:", lowest_low_ce)
    print(place_order(alice, TransactionType.Buy, symbol_ce, lowest_low_ce))

    # Get the lowest low and place orders for PE
    lowest_low_pe = fetch_lowest_low(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low PE Value:", lowest_low_pe)
    print(place_order(alice, TransactionType.Buy, symbol_pe, lowest_low_pe))

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter your user ID: 1141826
Enter your API key: nGhoLNREiDSHvBgjyGGQhltRljzoQX9XqvbLoAXoP6brZivMUymtMumvpK5bvqlkXyGcqVXbtmfanyXqixyRSo3DVSLdusxtTfYZIi9n0afG4ixXUOg0jS2uVOKm6kWY
Previous Trading Day: 2024-11-28 18:07:17.425863
Closing Price of Future Contract: 52329.4
CE Option Symbol: BANKNIFTY24DEC24C52300
PE Option Symbol: BANKNIFTY24DEC24P52400
Fetching current data for CE and PE options...
                datetime    open    high     low   close   volume
0    2024-11-29 09:15:59  820.00  905.00  820.00  893.95  22575.0
1    2024-11-29 09:16:59  893.65  911.95  889.65  901.45  23505.0
2    2024-11-29 09:17:59  903.50  914.95  898.00  914.95  16770.0
3    2024-11-29 09:18:59  912.00  920.00  905.70  910.00  12825.0
4    2024-11-29 09:19:59  908.00  910.35  886.65  900.85  12735.0
..                   ...     ...     ...     ...     ...      ...
370  2024-11

# Low backtesting

In [ ]:
from pya3 import *
from datetime import datetime, timedelta, time
from nsetools import Nse
import holidays
from google.colab import drive
import pandas as pd



def fetch_current_data(alice, symbol, from_time, to_time):
    """Fetch current data for the given symbol within the specified time range."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    current_data = alice.get_historical(instrument, from_time, to_time, interval='1')
    return current_data


def backtest(current_data, trigger_price, trade_type='Buy', target_points=150, stop_loss_points=75):
    """
    Backtesting function for CE and PE trades.
    :param current_data: DataFrame with current data for CE or PE.
    :param trigger_price: Entry price for the trade.
    :param trade_type: 'Buy' or 'Sell'.
    :param target_points: Profit target in points.
    :param stop_loss_points: Stop-loss in points.
    :return: Dictionary with backtest results.
    """
    if trade_type == 'Buy':
        target_price = trigger_price + target_points
        stop_loss_price = trigger_price - stop_loss_points
    elif trade_type == 'Sell':
        target_price = trigger_price - target_points
        stop_loss_price = trigger_price + stop_loss_points
    else:
        raise ValueError("Invalid trade type. Use 'Buy' or 'Sell'.")

    trade_open = False
    entry_time, exit_time, result = None, None, None

    for index, row in current_data.iterrows():
        timestamp = row['datetime']
        high = row['high']
        low = row['low']

        if not trade_open:
            if trade_type == 'Buy' and low <= trigger_price <= high:
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened (Buy) at {entry_time} with entry price {trigger_price}")
            elif trade_type == 'Sell' and low <= trigger_price <= high:
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened (Sell) at {entry_time} with entry price {trigger_price}")

        if trade_open:
            if trade_type == 'Buy' and low <= stop_loss_price:
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit (Buy) at {exit_time} with price {stop_loss_price}")
                break
            elif trade_type == 'Sell' and high >= stop_loss_price:
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit (Sell) at {exit_time} with price {stop_loss_price}")
                break
            elif trade_type == 'Buy' and high >= target_price:
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit (Buy) at {exit_time} with price {target_price}")
                break
            elif trade_type == 'Sell' and low <= target_price:
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit (Sell) at {exit_time} with price {target_price}")
                break

    if trade_open and result is None:
        result = 'Open'

    return {
        'Entry Time': entry_time,
        'Exit Time': exit_time,
        'Result': result,
        'Trade Type': trade_type
    }


# Other functions remain unchanged...

def main():
    # Step 1: Mount Google Drive
    drive.mount('/content/drive')

    # Get user credentials and initialize AliceBlue
    user_id, api_key = get_user_credentials()
    alice = initialize_alice(user_id, api_key)

    # Create an instance of Nse
    nse = Nse()

    # Get market holidays for the current year
    current_date = datetime.now()
    market_holidays = get_market_holidays(current_date.year)

    # Find the previous trading day
    previous_trading_day = find_previous_trading_day(current_date, market_holidays)
    print(f"Previous Trading Day: {previous_trading_day}")

    # Fetch the closing price of the future contract
    instrument = alice.get_instrument_by_symbol('NFO', 'BANKNIFTY24DEC24F')
    historical_data = fetch_historical_data(alice, instrument, previous_trading_day)

    if historical_data.empty:
        print("No historical data found for the future contract.")
        return

    last_close_price = historical_data['close'].iloc[-1]
    print(f"Closing Price of Future Contract: {last_close_price}")

    # Calculate strike prices based on the closing price of the future contract
    ce_strike_price, pe_strike_price = calculate_strike_prices(last_close_price)

    # Define the expiry date for options
    expiry_date = datetime(2024, 12, 24)

    # Construct option symbols
    symbol_ce, symbol_pe = construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price)
    print("CE Option Symbol:", symbol_ce)
    print("PE Option Symbol:", symbol_pe)


    # Define from_datetime, to_datetime, and interval for fetching highs and lows
    interval = "1"  # ["1", "D"]
    # Fetch and print current data for CE and PE from 9:15:29 to 15:17:29
    from_time = datetime.combine(current_date, time(9, 15, 29))
    to_time = datetime.combine(current_date, time(15, 17, 29))

    print("Fetching current data for CE and PE options...")
    ce_current_data = fetch_current_data(alice, symbol_ce, from_time, to_time)
    pe_current_data = fetch_current_data(alice, symbol_pe, from_time, to_time)

    print(ce_current_data)

     # Define from_datetime, to_datetime, and interval for fetching highs and lows
    from_datetime = datetime.combine(previous_trading_day, datetime.min.time())
    to_datetime = datetime.combine(previous_trading_day, datetime.max.time())
    interval = "D"

    # Get the highest high and place orders for CE
    highest_high_ce = fetch_highest_high(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Highest High CE Value:", highest_high_ce)
    print(place_order(alice, TransactionType.Sell, symbol_ce, highest_high_ce))

    # Get the highest high and place orders for PE
    highest_high_pe = fetch_highest_high(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Highest High PE Value:", highest_high_pe)
    print(place_order(alice, TransactionType.Sell, symbol_pe, highest_high_pe))

    # Get the lowest low and place orders for CE
    lowest_low_ce = fetch_lowest_low(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low CE Value:", lowest_low_ce)
    print(place_order(alice, TransactionType.Buy, symbol_ce, lowest_low_ce))

    # Get the lowest low and place orders for PE
    lowest_low_pe = fetch_lowest_low(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low PE Value:", lowest_low_pe)
    print(place_order(alice, TransactionType.Buy, symbol_pe, lowest_low_pe))

    print("Backtesting CE (Buy at low price)...")
    ce_buy_result = backtest(ce_current_data, lowest_low_ce, trade_type='Buy')

    print("Backtesting CE (Sell at high price)...")
    ce_sell_result = backtest(ce_current_data, highest_high_ce, trade_type='Sell')

    print("Backtesting PE (Buy at low price)...")
    pe_buy_result = backtest(pe_current_data, lowest_low_pe, trade_type='Buy')

    print("Backtesting PE (Sell at high price)...")
    pe_sell_result = backtest(pe_current_data, highest_high_pe, trade_type='Sell')

    # Print backtest results
    print("CE Buy Backtest Result:", ce_buy_result)
    print("CE Sell Backtest Result:", ce_sell_result)
    print("PE Buy Backtest Result:", pe_buy_result)
    print("PE Sell Backtest Result:", pe_sell_result)

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter your user ID: 1141826
Enter your API key: nGhoLNREiDSHvBgjyGGQhltRljzoQX9XqvbLoAXoP6brZivMUymtMumvpK5bvqlkXyGcqVXbtmfanyXqixyRSo3DVSLdusxtTfYZIi9n0afG4ixXUOg0jS2uVOKm6kWY
Previous Trading Day: 2024-11-28 18:15:38.454553
Closing Price of Future Contract: 52329.4
CE Option Symbol: BANKNIFTY24DEC24C52300
PE Option Symbol: BANKNIFTY24DEC24P52400
Fetching current data for CE and PE options...
                datetime    open    high     low   close   volume
0    2024-11-29 09:15:59  820.00  905.00  820.00  893.95  22575.0
1    2024-11-29 09:16:59  893.65  911.95  889.65  901.45  23505.0
2    2024-11-29 09:17:59  903.50  914.95  898.00  914.95  16770.0
3    2024-11-29 09:18:59  912.00  920.00  905.70  910.00  12825.0
4    2024-11-29 09:19:59  908.00  910.35  886.65  900.85  12735.0
..                   ...     ...     ...     ...     ...      ...
370  2024-11

# High Volatility

In [ ]:
from pya3 import *
from datetime import datetime, timedelta, time
from nsetools import Nse
import holidays
from google.colab import drive
import pandas as pd



def fetch_current_data(alice, symbol, from_time, to_time):
    """Fetch current data for the given symbol within the specified time range."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    current_data = alice.get_historical(instrument, from_time, to_time, interval='1')
    return current_data


def backtest(current_data, trigger_price, trade_type='Buy', target_points=150, stop_loss_points=75):
    """
    Backtesting function for CE and PE trades.
    :param current_data: DataFrame with current data for CE or PE.
    :param trigger_price: Entry price for the trade.
    :param trade_type: 'Buy' or 'Sell'.
    :param target_points: Profit target in points.
    :param stop_loss_points: Stop-loss in points.
    :return: Dictionary with backtest results.
    """
    if trade_type == 'Buy':
        target_price = trigger_price + target_points
        stop_loss_price = trigger_price - stop_loss_points
    elif trade_type == 'Sell':
        target_price = trigger_price - target_points
        stop_loss_price = trigger_price + stop_loss_points
    else:
        raise ValueError("Invalid trade type. Use 'Buy' or 'Sell'.")

    trade_open = False
    entry_time, exit_time, result = None, None, None

    for index, row in current_data.iterrows():
        timestamp = row['datetime']
        high = row['high']
        low = row['low']

        if not trade_open:
            if trade_type == 'Buy' and low <= trigger_price <= high:
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened (Buy) at {entry_time} with entry price {trigger_price}")
            elif trade_type == 'Sell' and low <= trigger_price <= high:
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened (Sell) at {entry_time} with entry price {trigger_price}")

        if trade_open:
            if trade_type == 'Buy' and low <= stop_loss_price:
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit (Buy) at {exit_time} with price {stop_loss_price}")
                break
            elif trade_type == 'Sell' and high >= stop_loss_price:
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit (Sell) at {exit_time} with price {stop_loss_price}")
                break
            elif trade_type == 'Buy' and high >= target_price:
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit (Buy) at {exit_time} with price {target_price}")
                break
            elif trade_type == 'Sell' and low <= target_price:
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit (Sell) at {exit_time} with price {target_price}")
                break

    if trade_open and result is None:
        result = 'Open'

    return {
        'Entry Time': entry_time,
        'Exit Time': exit_time,
        'Result': result,
        'Trade Type': trade_type
    }


# Other functions remain unchanged...

def main():
    # Step 1: Mount Google Drive
    drive.mount('/content/drive')

    # Get user credentials and initialize AliceBlue
    user_id, api_key = get_user_credentials()
    alice = initialize_alice(user_id, api_key)

    # Create an instance of Nse
    nse = Nse()

    # Get market holidays for the current year
    current_date = datetime.now()
    market_holidays = get_market_holidays(current_date.year)

    # Find the previous trading day
    previous_trading_day = find_previous_trading_day(current_date, market_holidays)
    print(f"Previous Trading Day: {previous_trading_day}")

    # Fetch the closing price of the future contract
    instrument = alice.get_instrument_by_symbol('NFO', 'BANKNIFTY24DEC24F')
    historical_data = fetch_historical_data(alice, instrument, previous_trading_day)

    if historical_data.empty:
        print("No historical data found for the future contract.")
        return

    last_close_price = historical_data['close'].iloc[-1]
    print(f"Closing Price of Future Contract: {last_close_price}")

    # Calculate strike prices based on the closing price of the future contract
    ce_strike_price, pe_strike_price = calculate_strike_prices(last_close_price)

    # Define the expiry date for options
    expiry_date = datetime(2024, 12, 24)

    # Construct option symbols
    symbol_ce, symbol_pe = construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price)
    print("CE Option Symbol:", symbol_ce)
    print("PE Option Symbol:", symbol_pe)


    # Define from_datetime, to_datetime, and interval for fetching highs and lows
    interval = "1"  # ["1", "D"]
    # Fetch and print current data for CE and PE from 9:15:29 to 15:17:29
    from_time = datetime.combine(current_date, time(9, 15, 29))
    to_time = datetime.combine(current_date, time(15, 17, 29))

    print("Fetching current data for CE and PE options...")
    ce_current_data = fetch_current_data(alice, symbol_ce, from_time, to_time)
    pe_current_data = fetch_current_data(alice, symbol_pe, from_time, to_time)

    print(ce_current_data)

     # Define from_datetime, to_datetime, and interval for fetching highs and lows
    from_datetime = datetime.combine(previous_trading_day, datetime.min.time())
    to_datetime = datetime.combine(previous_trading_day, datetime.max.time())
    interval = "D"

    # Get the highest high and place orders for CE
    highest_high_ce = fetch_highest_high(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Highest High CE Value:", highest_high_ce)
    print(place_order(alice, TransactionType.Buy, symbol_ce, highest_high_ce))

    # Get the highest high and place orders for PE
    highest_high_pe = fetch_highest_high(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Highest High PE Value:", highest_high_pe)
    print(place_order(alice, TransactionType.Buy, symbol_pe, highest_high_pe))

    # Get the lowest low and place orders for CE
    lowest_low_ce = fetch_lowest_low(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low CE Value:", lowest_low_ce)
    print(place_order(alice, TransactionType.Sell, symbol_ce, lowest_low_ce))

    # Get the lowest low and place orders for PE
    lowest_low_pe = fetch_lowest_low(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low PE Value:", lowest_low_pe)
    print(place_order(alice, TransactionType.Sell, symbol_pe, lowest_low_pe))

    print("Backtesting CE (Buy at low price)...")
    ce_buy_result = backtest(ce_current_data, lowest_low_ce, trade_type='Sell')

    print("Backtesting CE (Sell at high price)...")
    ce_sell_result = backtest(ce_current_data, highest_high_ce, trade_type='Buy')

    print("Backtesting PE (Buy at low price)...")
    pe_buy_result = backtest(pe_current_data, lowest_low_pe, trade_type='Sell')

    print("Backtesting PE (Sell at high price)...")
    pe_sell_result = backtest(pe_current_data, highest_high_pe, trade_type='Buy')

    # Print backtest results
    print("CE Buy Backtest Result:", ce_buy_result)
    print("CE Sell Backtest Result:", ce_sell_result)
    print("PE Buy Backtest Result:", pe_buy_result)
    print("PE Sell Backtest Result:", pe_sell_result)

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter your user ID: 1141826
Enter your API key: nGhoLNREiDSHvBgjyGGQhltRljzoQX9XqvbLoAXoP6brZivMUymtMumvpK5bvqlkXyGcqVXbtmfanyXqixyRSo3DVSLdusxtTfYZIi9n0afG4ixXUOg0jS2uVOKm6kWY
Previous Trading Day: 2024-11-28 18:17:59.159002
Closing Price of Future Contract: 52329.4
CE Option Symbol: BANKNIFTY24DEC24C52300
PE Option Symbol: BANKNIFTY24DEC24P52400
Fetching current data for CE and PE options...
                datetime    open    high     low   close   volume
0    2024-11-29 09:15:59  820.00  905.00  820.00  893.95  22575.0
1    2024-11-29 09:16:59  893.65  911.95  889.65  901.45  23505.0
2    2024-11-29 09:17:59  903.50  914.95  898.00  914.95  16770.0
3    2024-11-29 09:18:59  912.00  920.00  905.70  910.00  12825.0
4    2024-11-29 09:19:59  908.00  910.35  886.65  900.85  12735.0
..                   ...     ...     ...     ...     ...      ...
370  2024-11